---

<div align="center">
    <h3>Código para criação e inserção de dados de tabela .xls
    </h3>
    <p align="justify">
    O código que segue tem como objetivo extrair os dados contidos no arquivo .xls, averiguar as tipagens adequadas para cada dado, criar uma tabela com o mesmo nome do arquivo original e com cada uma das colunas, contendo já o tipo de dado. Esta versão ainda não apresenta capacidade de já pré-definir constraints, mas é uma feature prevista.
    </p>
</div>

---

In [ ]:
# Bibliotecas necessárias para o presente projeto
import psycopg2
import re
import pandas as pd
from unidecode import unidecode

In [ ]:
# Arquivo de credenciais necessárias para acessar banco de dados
credentials = pd.read_csv("../credentials/credentials.csv").iloc[0].to_dict()

In [ ]:
# Função para criar conexão e cursor
def conectar():
    host = credentials['host']
    dbname = credentials['database']
    user = credentials['user']
    password = credentials['password']
    sslmode = "require"

    conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)
    conexao = psycopg2.connect(conn_string) 
    print("Connection established")
    cursor = conexao.cursor()

    cursor = conexao.cursor()
    return conexao, cursor

# Função para executar uma query
def executar_query(cursor, query):
    cursor.execute(query)
    # Verificar se a consulta retorna resultados (SELECT)
    if cursor.description is not None:
        colunas = [desc[0] for desc in cursor.description]
        resultados = cursor.fetchall()
        return pd.DataFrame(resultados, columns=colunas)
    else:
        # Se não há resultados, retornar None ou outra indicação apropriada
        return None

# Função para fechar a conexão e o cursor
def fechar_conexao(conexao, cursor):
    cursor.close()
    conexao.close()
    print("Connection finished")

In [ ]:
# Carrega o arquivo .xls para um DataFrame
PATH = '../data_xls/Financial Sample.xlsx'
dados = pd.read_excel(PATH)

# Obtém o nome do arquivo (sem a extensão) e ajusta para ser utilizável como nome de tabela
nome_tabela = PATH.split('/')[-1].split('.')[0].lower().replace(" ","_")

# Obtém informações sobre as colunas e seus tipos
info_colunas = dados.dtypes.reset_index()
info_colunas.columns = ['nome_coluna', 'tipo']

In [ ]:
# Adequa nome das colunas caso haja espaço antes ou depois
info_colunas['nome_coluna'] = info_colunas['nome_coluna'].str.strip()

# Adequação dos nomes das colunas.
info_colunas['novo_nome_coluna'] = info_colunas['nome_coluna'].apply(lambda x: re.sub(r'\W+', '_', unidecode(str(x)).lower()))

# Transforma o nome dos tipos em string para devida identificação
info_colunas['tipo']  = info_colunas['tipo'].astype('str')

# Converte os tipos pandas para tipos SQL
mapeamento_tipos_sql = {
    'int64': 'INTEGER',
    'float64': 'FLOAT',
    'object': 'TEXT',
    'datetime64[ns]': 'TIMESTAMP'
}

info_colunas['tipo_sql'] = info_colunas['tipo'].map(mapeamento_tipos_sql)

In [ ]:
# Dicionário de conversão de nomes da tabela .xls para tabela no SQL
de_para = dict(zip(info_colunas['nome_coluna'], info_colunas['novo_nome_coluna']))

# Usa de_para para renomear coluna de dados
dados.rename(columns=de_para, inplace=True)

In [ ]:
# Construção de query para criação da tabela referente ao arquivo em questão
query = f'''CREATE TABLE IF NOT EXISTS {nome_tabela} ( \n'''
for i, j, coluna, tipo in info_colunas.values:
    if coluna != info_colunas["novo_nome_coluna"].iloc[-1]:
        query += f" {coluna} {tipo},\n"
    else:
        query += f" {coluna} {tipo}\n);"

## Cria conexão
conexao, cursor = conectar()

# Executa a query
df = executar_query(cursor, query)

# Comita possível criação de nova tabela
conexao.commit()

# Fecha conexão
fechar_conexao(conexao, cursor)

In [ ]:
# Inicia conexão
conexao, cursor = conectar()

# Inserção apenas das primeiras 500 linhas
for indice, linha in dados.iloc[:500,:].astype(str).iterrows():
    linha_como_tupla = str(tuple(linha))
    colunas = str(tuple(dados.columns)).replace("'","")
    query = f'INSERT INTO {(nome_tabela)} {colunas} VALUES {linha_como_tupla}'
    executar_query(cursor, query)

# Comita inserções realizadas
conexao.commit()

# Fecha conexão
fechar_conexao(conexao, cursor)